# Lab 05-1: Multiclass Logistic Regression
## Exercise: Predicting Iris Species

### Prepare IRIS Dataset

In [1]:
import numpy as np
import pandas as pd

from sklearn.datasets import load_iris

iris = load_iris()

# iris.data contains four column
#   sepal length (cm) / sepal width (cm) / petal length (cm) / petal width (cm)
# iris.target contains one column
#   species of (0,1,2) = (setosa, versicolor, virginica)
iris_df = pd.DataFrame(data= iris.data, columns= iris.feature_names)
iris_tf = pd.DataFrame(data= iris.target, columns= ['species'])

vX = iris_df.copy()
vY = iris_tf['species']

# Chnage dataset from pandas to numpy
vX = vX.to_numpy()
vY = vY.to_numpy()

### Presenting Dataset Samples

In [2]:
iris_df = pd.concat([iris_df, iris_tf], axis= 1)
iris_df.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333,1.000000
std,0.828066,0.435866,1.765298,0.762238,0.819232
min,4.300000,2.000000,1.000000,0.100000,0.000000
25%,5.100000,2.800000,1.600000,0.300000,0.000000
50%,5.800000,3.000000,4.350000,1.300000,1.000000
75%,6.400000,3.300000,5.100000,1.800000,2.000000
max,7.900000,4.400000,6.900000,2.500000,2.000000


In [3]:
print(vY)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


Splitting Data for Training and Testing, then Preparing one-hot labeled ground truth

In [4]:
# We can use train_test_split from sklearn
from sklearn.model_selection import train_test_split

# Splitting dataframe into train & test
X_train, X_test, y_train_num, y_test = train_test_split(vX, vY, test_size= 0.20, random_state= 101)

# species are 0 for setosa, 1 for versicolor, and 2 for virginica
# make one-hot training sequence from numerical category sequence
n_train = y_train_num.shape[0]
y_train = np.zeros((n_train, 3))
for i in range(n_train):
    y_train[i, y_train_num[i]] = 1

### Multiclass Logistic Regression

$$h(x^{(i)}) = \text{softmax} (Wx^{(i)} + b), \qquad
J = -{1 \over n} \sum_{i=1}^{n} \left(y (Wx^{(i)} + b) - \log(1+e^{Wx^{(i)} + b}) \right)$$

$${\partial J \over \partial W} = {1 \over n} \sum_{i=1}^{n} \left(\left(y - h(x^{(i)})\right) \cdot x_j^{(i)}\right), \qquad
{\partial J \over \partial b} = {1 \over n} \sum_{i=1}^{n} \left(y - h(x^{(i)})\right)$$

Define Logistic Functions

In [5]:
# define softmax. Assume (b, s)
def softmax(x):
    x = np.exp(x)                            # (b, s)
    xsum = np.sum(x, axis=-1, keepdims=True) # (b, s)
    return x / xsum                          # (b, s)

Define Model Class

In [25]:
class myLinearRegression:
    def __init__(self, n_out, n_in):
        self.wegt = np.zeros((n_out, n_in))
        self.bias = np.zeros((n_out))

Create a model and check the matrix dimensions

In [26]:
n_classes = 3
n_inputs  = 4

m = myLinearRegression(n_classes, n_inputs)

print(m.wegt.shape, m.bias.shape)
print(X_train.shape, y_train.shape)

(3, 4) (3,)
(120, 4) (120, 3)


Training mult-class Model with Linear Regression

In [27]:
# define learning rate alpha, and number of epochs
# n_train = number of samples in train dataset
alpha = 0.01
n_epochs = 1000

for epoch in range(n_epochs):
    ### START CODE HERE ###
    # Be careful with matrix dimensions

    # forward path
    y_lin  = X_train @ m.wegt.T + m.bias         # Linear Prediction
    y_prob = softmax(y_lin)         # Find Class Probability (softmax)

    # update weights
    y_diff = y_prob - y_train         # Calculate Differences
    m.wegt = m.wegt - alpha * y_diff.T @ X_train / X_train.shape[0]         # Update Weights (class, input)
    m.bias = m.bias - alpha * np.mean(y_diff)         # Update Bias (class,)

    ### END CODE HERE ###

    # Print loss values
    if ((epoch+1)%100==0):
        ### START CODE HERE ###

        y_lin  = X_train @ m.wegt.T + m.bias     # Linear Prediction
        y_prob = softmax(y_lin)     # Find the Class Probability
        loss_J = -np.sum(y_train * np.log(y_prob))     # Calculate Multi-class CE Loss J

        ### END CODE HERE ###
        print('Epoch: %4d,  loss: %10.8f' % (epoch+1, loss_J))

Epoch:  100,  loss: 84.20507461
Epoch:  200,  loss: 69.22172093
Epoch:  300,  loss: 61.74194711
Epoch:  400,  loss: 56.97043976
Epoch:  500,  loss: 53.48235784
Epoch:  600,  loss: 50.71927632
Epoch:  700,  loss: 48.41813483
Epoch:  800,  loss: 46.43794963
Epoch:  900,  loss: 44.69554570
Epoch: 1000,  loss: 43.13802930


Evaluate Model Performance

In [28]:
def my_predict(m, X_test):
    ### START CODE HERE ###

    y_lin  = X_test @ m.wegt.T + m.bias         # Linear Prediction
    y_prob = softmax(y_lin)         # Find the Class Probability
    y_pred = np.argmax(y_prob, axis=1)         # Make Prediction

    ### END CODE HERE ###
    return y_pred

from sklearn.metrics import accuracy_score

y_pred = my_predict(m, X_test)

print(y_test)
print(y_pred)

accuracy_score(y_pred, y_test)

[0 0 0 2 1 2 1 1 2 0 2 0 0 2 2 1 1 1 0 2 1 0 1 1 1 1 1 2 0 0]
[0 0 0 2 1 2 1 1 2 0 2 0 0 2 2 1 1 1 0 2 1 0 1 1 1 1 1 2 0 0]


1.0

Linear Regression from scikit-learn

In [29]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

# Training/Fitting the Model
lr.fit(X_train, y_train_num)

# Making Predictions
s_pred = lr.predict(X_test)

accuracy_score(s_pred, y_test)

1.0

### Test Model with a random sample


In [30]:
idx = np.random.randint(X_test.shape[0])
test_in = np.expand_dims(X_test[idx], axis=0)

species = ['setosa', 'versicolor', 'virginica']

y_pred = my_predict(m, test_in)
s_pred = lr.predict(test_in)

print('My Prediction for Iris Species:', species[y_pred[0]])
print('SK Prediction for Iris Species:', species[s_pred[0]])
print('True Iris Species is:', species[y_test[idx]])

My Prediction for Iris Species: virginica
SK Prediction for Iris Species: virginica
True Iris Species is: virginica
